In [1]:
import ray
from multiprocessing import Pool

In [2]:
#tag::bad_ser_example[]
pool = Pool(5)

def special_business(x):
    def inc(y):
        return y + x
    return pool.map(inc, range(0, x))
ray.util.inspect_serializability(special_business)
#end::bad_ser_example[]

Checking Serializability of <function special_business at 0x7f1362e2bd30>
!!! FAIL serialization: pool objects cannot be passed between processes or pickled
Detected 1 global variables. Checking serializability...
    Serializing 'pool' <multiprocessing.pool.Pool state=RUN pool_size=5>...
    !!! FAIL serialization: pool objects cannot be passed between processes or pickled
        Serializing 'Process' <function Pool.Process at 0x7f1362e1e940>...
        Serializing '_get_tasks' <function Pool._get_tasks at 0x7f1362e1b820>...
        Serializing '_get_worker_sentinels' <function Pool._get_worker_sentinels at 0x7f1362e1ec10>...
        Serializing '_handle_results' <function Pool._handle_results at 0x7f1362e1b790>...
        Serializing '_handle_tasks' <function Pool._handle_tasks at 0x7f1362e1b700>...
        Serializing '_help_stuff_finish' <function Pool._help_stuff_finish at 0x7f1362e1baf0>...
        Serializing '_join_exited_workers' <function Pool._join_exited_workers at 0x7f136

(False,
 {FailTuple(notifier [obj=<multiprocessing.queues.SimpleQueue object at 0x7f1362e5df70>, parent={}])})

In [3]:
#tag::ex_pydev_charm[]
@ray.remote
class Bloop():
    
    def __init__(self, dev_host):
        import pydevd_pycharm
        # Requires ability to connect to dev from prod.
        try:
            pydevd_pycharm.settrace(dev_host, port=7779, stdoutToServer=True, stderrToServer=True)
        except ConnectionRefusedError:
            print("Skipping debug")
            pass
    
    def dothing(x):
        return x + 1
#end::ex_pydev_charm[]

In [4]:
bloop_actor = Bloop.remote("localhost")

Usage stats collection is enabled. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2022-09-27 08:40:18,875	INFO worker.py:1510 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(Bloop pid=1146622) 0.05s - Could not connect to localhost: 7779


(Bloop pid=1146622) Skipping debug


(Bloop pid=1146622) Traceback (most recent call last):
(Bloop pid=1146622)   File "/home/holden/.local/lib/python3.8/site-packages/_pydevd_bundle/pydevd_comm.py", line 507, in start_client
(Bloop pid=1146622)     s.connect((host, port))
(Bloop pid=1146622) ConnectionRefusedError: [Errno 111] Connection refused
